<a href="https://colab.research.google.com/github/sqbitegh/Colabs/blob/main/VerifyModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Verify Model**

In [19]:
#configuration

#pl
language_abbrev = "pl"
language_full = "Polish"
dataset_in_hub_path = "ambind/preprocessed_commonvoice_pl"

#en
#language_abbrev = "en"
#language_full = "English"
#dataset_in_hub_path = "ambind/preprocessed_commonvoice_en_1proc"
#dataset_in_hub_path = "ambind/preprocessed_commonvoice_en"


model_name = "openai/whisper-tiny"

In [20]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Nov 18 17:42:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0              28W /  70W |   2169MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
!pip install --upgrade --quiet pip
!pip install --quiet datasets[audio]
!pip install evaluate
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.6 MB/s eta 0:00:00


Linking the notebook to the Hub is straightforward - it simply requires entering your Hub authentication token when prompted. Find your Hub authentication token [here](https://huggingface.co/settings/tokens):

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [21]:
from transformers import WhisperForConditionalGeneration
from transformers import WhisperTokenizer
from transformers import WhisperFeatureExtractor
from transformers import WhisperProcessor

firstrun = True
model = WhisperForConditionalGeneration.from_pretrained(model_name)
processor = WhisperProcessor.from_pretrained(model_name, language=language_full, task="transcribe")
feature_extractor = processor.feature_extractor
#feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
tokenizer = processor.tokenizer


In [32]:
#cached dataset loading
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()
dataset_ready = False
try:
        #raise FileNotFoundError #needed to reload from source repo as otherwise hfhub could provide cached content
        #data_files = {"train": f"{dataset_in_hub_path}/train*.parquet", "test": f"{dataset_in_hub_path}/test*.parquet"}
        #data_files = { "test": f"{dataset_in_hub_path}/test*.parquet"}
        #for en 1%
        #common_voice["train"] = load_dataset(path=dataset_in_hub_path, split ="train")
        #common_voice["test"] = common_voice["train"]

        #for pl
        common_voice["test"] = load_dataset(path=dataset_in_hub_path, split ="test", streaming=True)
        common_voice["train"] = common_voice["test"]
        dataset_ready = True
       # print("dataset loaded from " + dataset_in_hub_path + " content " + common_voice["train"])
except FileNotFoundError:
        print(f"Dataset '{dataset_in_hub_path}' not found on the Hugging Face Hub.")
except Exception as e:
        print(f"An unexpected error occurred: {e}")

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

In [33]:

import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

model.generation_config.language = language_full
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None

metric = evaluate.load("wer")

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

from IPython.display import display

print("common_voice[test]")
display(common_voice["test"])

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
     output_dir="./whisper-verify",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=4e-50, #make it only flow through
    warmup_steps = 500 if firstrun else 2, #debug  warmup_steps=50,
    max_steps=15,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    save_strategy="no",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    #save_steps=1000, #debug
    eval_steps=10,
    logging_steps=100,
    report_to=["tensorboard"],
    load_best_model_at_end=False,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

from transformers import Seq2SeqTrainer
from datasets import Dataset

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["train"], #eval_dataset=Dataset.from_dict(common_voice["test"][:100]),  test over training in some cases
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

common_voice[test]


IterableDataset({
    features: ['input_features', 'labels'],
    num_shards: 16
})

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-33-d05d4b638d91>:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
max_steps is given, it will override any value given in num_train_epochs


In [34]:
trainer.train()

Step,Training Loss,Validation Loss,Wer
10,No log,1.529188,62.573099


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=15, training_loss=1.6140268961588542, metrics={'train_runtime': 2168.9126, 'train_samples_per_second': 0.111, 'train_steps_per_second': 0.007, 'total_flos': 5908532428800000.0, 'train_loss': 1.6140268961588542, 'epoch': 1.0})